In [ ]:
import pandas as pd
import numpy as np
import selenium as sel
from matplotlib import pyplot as plt

In [ ]:
from selenium.webdriver.common.by import By
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException

In [ ]:
import networkx as nx

In [ ]:
url = "https://www.un.org/en"
pages_limit = 50
conn_limit = 3

In [ ]:
base_url = url.replace("https://","").replace("http://","")[
    0:url.replace("https://","").replace("http://","").find("/")]
counter = 0
links_matrix = {}
links_ref = {url: 'Home'}

In [ ]:
driver = webdriver.Safari()

In [ ]:
def parse_links(url, counter):
    try:
        driver.get(url)
        ignored_exceptions=(NoSuchElementException,StaleElementReferenceException,)
        WebDriverWait(driver, 30, ignored_exceptions=ignored_exceptions).until(
            EC.presence_of_element_located((By.XPATH, "//a[@href]")))
        links = driver.find_elements_by_xpath("//a[@href]")
        if counter==0:
            driver.implicitly_wait(5)
        links = [l for l in links if l.get_attribute("href") not in \
                 ["", "javascript:void(0)", "javascript:void(0);"] and \
                 l.get_attribute("href") not in links_ref.keys() and base_url in l.get_attribute("href") and \
                 ".pdf" not in l.get_attribute("href")]
        links = links[0:min(conn_limit, len(links))]
        if len(links)>0:
            links_matrix[url] = {} 
            for link in links:
                counter += 1
                if counter>pages_limit:
                    return None
                else:
                    print("Pages parsed:", counter)
                    links_ref[link.get_attribute("href")] = link.text.strip()
                    links_matrix[url][link.get_attribute("href")] = 1
                    parse_links(link.get_attribute("href"), counter)
        else:
            pass
    except TimeoutException:
        print("Timeout. Page skipped.")
        pass
    return None

In [ ]:
parse_links(url, 0)

In [ ]:
pd.DataFrame(links_matrix)

In [ ]:
G = nx.DiGraph()
for x in links_matrix.keys():
    for y in links_matrix[x].keys():
        G.add_edge(links_ref[x][:20], links_ref[y][:20])

In [ ]:
links_ref

In [ ]:
plt.figure(figsize=(18,18))
nx.draw(G, with_labels=True)
plt.show()